# Temporal Eigenstate Networks - Quickstart

This notebook demonstrates how to use Temporal Eigenstate Networks (TEN) for sequence modeling tasks.

## Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from model import TemporalEigenstateNetwork, TemporalEigenstateConfig
from train import Trainer
from eval import Evaluator

## Create a Model

In [ ]:
# Configure the model
config = TemporalEigenstateConfig(
    d_model=256,
    n_heads=8,
    n_layers=4,
    d_ff=1024,
    max_seq_len=512,
    dropout=0.1,
)

# Create the model
model = TemporalEigenstateNetwork(config)
print(f"Model created with {sum(p.numel() for p in model.parameters())} parameters")

## Forward Pass Example

In [ ]:
# Create dummy input
batch_size = 4
seq_len = 64
d_model = 256

x = torch.randn(batch_size, seq_len, d_model)

# Forward pass
model.eval()
with torch.no_grad():
    output = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

## Efficiency Benchmark

In [ ]:
evaluator = Evaluator(model, device='cpu')

# Measure efficiency across different sequence lengths
seq_lengths = [32, 64, 128, 256, 512]
results = evaluator.measure_efficiency(
    seq_lengths=seq_lengths,
    batch_size=8,
    d_model=256,
    num_runs=5,
)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(results['seq_lengths'], results['forward_time'], marker='o')
ax1.set_xlabel('Sequence Length')
ax1.set_ylabel('Forward Time (s)')
ax1.set_title('Computational Efficiency')
ax1.grid(True)

if any(m > 0 for m in results['memory_usage']):
    ax2.plot(results['seq_lengths'], results['memory_usage'], marker='o', color='orange')
    ax2.set_xlabel('Sequence Length')
    ax2.set_ylabel('Memory Usage (MB)')
    ax2.set_title('Memory Efficiency')
    ax2.grid(True)

plt.tight_layout()
plt.show()

## Training Example

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create synthetic dataset
num_samples = 1000
X_train = torch.randn(num_samples, 64, 256)
y_train = torch.randn(num_samples, 64, 256)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Setup trainer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

trainer = Trainer(model, optimizer, criterion, device='cpu')

# Train for a few epochs
history = trainer.fit(train_loader, epochs=3)

# Plot training loss
plt.figure(figsize=(8, 4))
plt.plot(history['train_loss'], marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

## Conclusion

This notebook demonstrated:
- Creating a TEN model
- Running forward passes
- Measuring computational efficiency
- Training the model

For more examples, see the paper and benchmarks.